In [ ]:
1. 根据LFM的embedding生成输入
2. 将数据输入到FM进行训练

In [4]:
# 处理数据，生成输入
import pandas as pd
from lfm_new_data import LFM
import numpy as np
model=pd.read_pickle('UI.pkl')
user_embeddings=model.user_embeddings.weight.data.numpy()
item_embeddings=model.item_embeddings.weight.data.numpy()

data_path="hin/data/u.data"
data_fields = ['user_id', 'item_id', 'rating', 'timestamp']
# all data file
data_df = pd.read_table(data_path, names=data_fields)

# get user number
n_users = max(data_df['user_id'].values)
# get item number
n_items = max(data_df['item_id'].values)

data_df['user_id']-=1
data_df['item_id']-=1


X=[]
for i in range(data_df.shape[0]):
    x=data_df.iloc[i]
    X.append(np.concatenate((user_embeddings[x['user_id']],item_embeddings[x['item_id']])))
    
X=np.array(X)

y=data_df.rating.values
y=y.reshape(-1,1)
y=y.astype(np.float32)

X_tensor,  y_tensor = torch.from_numpy(X).to(device), torch.from_numpy(
    y).to(device)

train_num = int(X_tensor.shape[0] * 0.9)
train_set = TensorDataset(X_tensor[:train_num], y_tensor[:train_num])
valid_set = TensorDataset(X_tensor[train_num:], y_tensor[train_num:])

loaders = {'train': DataLoader(train_set, batch_size=batch_size, shuffle=True),
           'valid': DataLoader(valid_set, batch_size=batch_size, shuffle=False)}

input_size = X_tensor.shape[1]

NameError: name 'torch' is not defined

In [76]:
import pandas as pd
pd.to_pickle(loaders,'loaders.pkl')

In [74]:
import random
import math
import pandas as pd
import numpy as np
import math
import torch
import torch.nn as nn
import torch.utils.data as data
from tqdm import tqdm
from utils import Interactions
import os
np.random.seed(1024)
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import roc_auc_score
# for reproducibility
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

# To compute probalities
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class FMG(torch.nn.Module):
    def __init__(self, input_size=2, n_factors=10, lr=0.01, weight_decay=0.):
        super(FMG, self).__init__()

        # get factor number
        # Initially we fill V with random values sampled from Gaussian distribution
        # NB: use nn.Parameter to compute gradients
        self.V = nn.Parameter(torch.randn(input_size, n_factors),requires_grad=True)
        self.lin = nn.Linear(input_size, 1)

        self.optimizer = torch.optim.Adam(self.parameters(),
                                   lr=lr, weight_decay=weight_decay)

    def forward(self, x):
        out_1 = torch.matmul(x, self.V).pow(2).sum(1, keepdim=True)  # S_1^2
        out_2 = torch.matmul(x.pow(2), self.V.pow(2)).sum(1, keepdim=True)  # S_2

        out_inter = 0.5 * (out_1 - out_2)
        out_lin = self.lin(x)
        out = out_inter + out_lin

        return out

    def fit(self, loaders, epochs=5):
        # training cycle
        best_score = 0.
        for epoch in range(epochs):
            losses = {'train': 0., 'valid': 0}

            for phase in ['train', 'valid']:

                if phase == 'train':
                    self.train()
                else:
                    self.eval()

                for batch_x, batch_y in loaders[phase]:
                    self.optimizer.zero_grad()
                    out = self.forward(batch_x)
#                     loss = nn.BCEWithLogitsLoss()(out, batch_y)
                    loss = nn.MSELoss(reduction='sum')(out, batch_y)
                    losses[phase] += loss.item() * batch_x.size(0)

                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            loss.backward()
                            #                             scheduler.step()
                            self.optimizer.step()

                losses[phase] /= len(loaders[phase].dataset)

            # after each epoch check if we improved roc auc and if yes - save model
            with torch.no_grad():
                model.eval()

                y_pred,y_true = [],[]
                for batch_x, batch_y in loaders['valid']:
                    out = self.forward(batch_x)
                    preds = sigmoid(out.cpu().numpy())
                    y_pred += preds.tolist()
                    y_true += batch_y.tolist()
#                 epoch_score = roc_auc_score(np.array(y_true), np.array(y_pred))
#                 embed()
                epoch_score=sum([(y[0] - x[0]) ** 2 for x, y in zip(y_true, y_pred)]) / len(y_pred)


            if ((epoch + 1) % 1) == 0:
                print(
                    f'epoch {epoch + 1} train loss: {losses["train"]:.3f} valid loss {losses["valid"]:.3f} valid mse {epoch_score:.3f}')
        return


In [51]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x21f0edae390>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x21f0edae128>}

In [75]:
model = FMG(input_size)
model.fit(loaders,15)

epoch 1 train loss: 179.141 valid loss 32.662 valid mse 7.833
epoch 2 train loss: 30.039 valid loss 27.462 valid mse 7.818
epoch 3 train loss: 27.764 valid loss 30.055 valid mse 7.858
epoch 4 train loss: 26.936 valid loss 27.466 valid mse 7.819
epoch 5 train loss: 26.829 valid loss 27.343 valid mse 7.798
epoch 6 train loss: 26.917 valid loss 25.968 valid mse 7.807
epoch 7 train loss: 26.732 valid loss 27.032 valid mse 7.805
epoch 8 train loss: 26.787 valid loss 26.377 valid mse 7.811
epoch 9 train loss: 26.888 valid loss 26.035 valid mse 7.818
epoch 10 train loss: 26.736 valid loss 26.214 valid mse 7.816
epoch 11 train loss: 26.753 valid loss 26.125 valid mse 7.812
epoch 12 train loss: 26.949 valid loss 25.893 valid mse 7.816
epoch 13 train loss: 26.837 valid loss 27.020 valid mse 7.830
epoch 14 train loss: 26.805 valid loss 26.078 valid mse 7.820
epoch 15 train loss: 26.763 valid loss 27.150 valid mse 7.806
